In [1]:
import pandas as pd

In [2]:
ani_report_uri = 'https://ftp.ncbi.nlm.nih.gov/genomes/ASSEMBLY_REPORTS/ANI_report_prokaryotes.txt'
ani_report = pd.read_csv(ani_report_uri, sep='\t', low_memory=False)

In [3]:
assembly_summary_uri ='https://ftp.ncbi.nlm.nih.gov/genomes/refseq/assembly_summary_refseq.txt'
assembly_summary = pd.read_csv(assembly_summary_uri, sep='\t', header=1, low_memory=False)

In [45]:
species_name = 'Shigella boydii'
assembly_level = [
    'Complete Genome',
#     'Chromosome',
#     'Scaffold',
#     'Contig'
]

In [46]:
taxonomy_check_ok = ani_report[
    (ani_report['taxonomy-check-status']=='OK')&
    (ani_report['species-name']==species_name)
]

In [47]:
specific_species_assembly = assembly_summary[
    (assembly_summary.organism_name.str.startswith(species_name))&
    (assembly_summary.assembly_level.isin(assembly_level))&
    (assembly_summary.excluded_from_refseq.isna())
]

In [48]:
reference_assemblies = specific_species_assembly[specific_species_assembly['gbrs_paired_asm'].isin(taxonomy_check_ok['# genbank-accession'])]

In [49]:
assembly_accessions = reference_assemblies.gbrs_paired_asm.to_list()

In [50]:
len(assembly_accessions)

11

In [51]:
import subprocess
from concurrent.futures import ThreadPoolExecutor

In [52]:
outdir = "/media/Central_Lab_Storage/NcbiASM/Shigella_boydii/RefSeq"

def fn(accession):
    cmd = ['conda', 'run', '-n', 'notebook3.8', 'python', '/home/chen1i6c04/download_ncbi_assembly.py', '-o', outdir]
    cmd.append(accession)
    subprocess.run(cmd)

In [53]:
with ThreadPoolExecutor(16) as executor:
    executor.map(fn, assembly_accessions, chunksize=50)